## Dimensionality Reduction
##### Dimensionality Reduction Algorithms
- Linear Algebra Methods
- Manifold Learning Methods: seek a lower-dimensional projection of high dimensional input that captures the salient properties of the input data

In [1]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sklearn
print(sklearn.__version__)
## 0.23.0 이상인 버전

1.2.2


#### 모의실험용 데이터 생성: 분류분석

In [3]:
from sklearn.datasets import make_classification
설명변수, 반응변수 = make_classification(n_samples=1000,n_features=5, n_informative=5, n_redundant=0, random_state=0 )
##데이터는 100개, 칼럼은 5개(모든 변수가 y(반응변수)에 영향을 주는 변수), 다중공선성= 0

In [4]:
import statsmodels.api as sm
##with statsmodels
설명변수 = sm.add_constant(설명변수) # adding a contant 절편이 없는 모델이기 때문에 상수를 추가함 
print(sm.Logit(반응변수, 설명변수).fit().summary()) ## fit 한 것의 구조를 요약하여 나타냄 

Optimization terminated successfully.
         Current function value: 0.229467
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      994
Method:                           MLE   Df Model:                            5
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                  0.6689
Time:                        16:17:07   Log-Likelihood:                -229.47
converged:                       True   LL-Null:                       -693.15
Covariance Type:            nonrobust   LLR p-value:                3.185e-198
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8778      0.225      3.898      0.000       0.436       1.319
x1             0.3045      0.

In [5]:
##표본크기=1000, 설명변수개수 = 20, 설명력이 있는 변수 =10, 불필요한 변수=10
설명변수, 반응변수 = make_classification(n_samples=1000,n_features=20, n_informative=5, n_redundant=10,random_state=0 )
설명변수 = sm.add_constant(설명변수)#adding a constant
print(sm.Logit(반응변수, 설명변수).fit().summary())## 불필요한 변수를 가져온것 빼고 나머지는 위랑 똑같음
## 아래 표를 가지고 설명하기에는 어려움이 있음 

         Current function value: 0.243352
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      989
Method:                           MLE   Df Model:                           10
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                  0.6489
Time:                        16:19:32   Log-Likelihood:                -243.35
converged:                      False   LL-Null:                       -693.14
Covariance Type:            nonrobust   LLR p-value:                7.860e-187
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8979      0.217      4.141      0.000       0.473       1.323
x1             0.1679      0.122      1.376      0.169      -0.07

C:\Users\SM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


#### 차원축소 방법을 적용한 자료에 대한 로지스틱 회귀분석
Repeated stratified 10-fold cross-validation 방법을 통해 성능비교

In [7]:
from numpy import mean ## 비교하는 방식으로 사용됨 , 데이터가 많은 경우에는 train + test셋으로 나누지만 데이터가 많이 없는 경우에는 k개의 칸으로 나눠서 한칸을 테스트로 사용
from numpy import std ## 표준편차 계산 
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression ## 예측 모델을 사용하기 위해 로지스틱 리그레션 방식을 사용함 
## stats 모델은 통계적인 값들을 나타내기 위해 사용하는 것이고 sklearn 은 "예측" 하기위해서 사용하는 방식임 

In [8]:
# 모형 설정
분석방법 = LogisticRegression()

#원자료에 대한 평가
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1) ##10개로 자르고 3번 반복하여 사용함 
점수 = cross_val_score(분석방법, 설명변수, 반응변수, scoring='accuracy', cv=cv, n_jobs=-1)## n_jobs: 모든 cpu를 다 사용하라!
print('Accuracy: %.3f (%.3f)' % (mean(점수), std(점수)))

Accuracy: 0.902 (0.026)


In [10]:
##PCA: 서로 독립이 아닌데이터 들을 묶어서 독립인 데이터처럼 만들어줌 
## 파이프라인 : 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

PCA(n_components=5).fit_transform(설명변수) ## PCA에서 5개의 칼럼들로 뽑아옴 -> 각 한줄씩 분석을 함 

array([[-3.49116061, -3.18790272,  1.12960961,  0.54013697,  0.02654012],
       [-0.10276923, -3.50457665, -0.72794009,  0.05346297, -0.91530089],
       [-3.0979493 ,  3.53395139, -2.76254129, -4.2934772 , -0.83307225],
       ...,
       [-2.14847104,  6.97758979,  0.91435403, -0.2051838 , -0.81127541],
       [ 2.44834879, -0.22556633,  4.41572754,  0.97962709,  1.22745587],
       [-3.67345152, -3.69101242, -1.22137635, -2.54358137, -0.43784689]])

In [11]:
과정 = [('pca', PCA(n_components=5)), ('m',LogisticRegression() )] ## 뽑힌 데이터들을 로지스틱 회귀분석에 적용함
## pca 방법을 적용함(차원 축소). 이것을 로지스틱 회귀분석에 적용함
모형 = Pipeline(steps=과정)## step이란 값에 "과정"을 대입함
모형
## pca모형에 "설명변수"를 집어넣으면 p값이 나옴 이후 "반응변수"랑 "p"값을 "로지스틱 회귀분석"에 넣어서 적용함 

Pipeline(steps=[('pca', PCA(n_components=5)), ('m', LogisticRegression())])

In [12]:
점수 = cross_val_score(모형, 설명변수, 반응변수,scoring='accuracy', cv = cv, n_jobs=-1 )
print('Accuracy: %.3f (%.3f)' % (mean(점수), std(점수)))

Accuracy: 0.906 (0.028)


In [13]:
from sklearn.decomposition import TruncatedSVD
TruncatedSVD(n_components=5).fit_transform(설명변수) ## 5개의 데이터들을 사용 -> 데이터들에 대한 추정값을 구할 수 있음
## 위와는 다른 방식으로 5개의 칼럼 들을 뽑아줌 (차원축소)

array([[-3.70343229, -3.98650213, -1.37265822, -0.63471761,  1.03247114],
       [-0.32235725, -4.55947547, -0.71336881,  0.67398658, -0.48779025],
       [-3.28594421, -0.12982111,  5.53813371, -1.45593703, -3.56446996],
       ...,
       [-2.23701309,  4.47584669,  4.24237147, -1.62495065,  0.73184243],
       [ 2.28298104, -1.0570723 , -1.26324642, -3.26202823,  3.29992018],
       [-3.92271469, -5.39950294,  0.2761539 , -0.79135582, -2.22434667]])

In [14]:
과정 = [('svd', TruncatedSVD(n_components=5)), ('m', LogisticRegression())]
모형 = Pipeline(steps=과정)
점수 = cross_val_score(모형, 설명변수, 반응변수, scoring='accuracy',cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(점수), std(점수)))

Accuracy: 0.906 (0.027)


In [22]:
from sklearn.manifold import Isomap
Isomap(n_components=5).fit_transform(설명변수)## 5개의 변수를 사용함 

array([[-7.76254864, -0.66304379,  1.80835807,  0.62572071, -0.45239671],
       [-2.7732265 , -5.77186012,  0.65549934, -0.73085225, -2.05136113],
       [-1.32547897, 12.58390657, -5.94061377, -0.55662981, -2.72985668],
       ...,
       [ 9.62942053, 15.28561819, -0.58394826,  3.80816428,  1.22223658],
       [ 6.0879797 , -0.23694135, 11.87014937, -3.25689988,  3.55412476],
       [-9.50504788, -4.15974532,  0.46826841, -3.20340085, -4.754981  ]])

In [23]:
과정 = [('iso', Isomap(n_components=5)), ('m', LogisticRegression())]
모형 = Pipeline(steps=과정)
점수 = cross_val_score(모형, 설명변수, 반응변수, scoring='accuracy',cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(점수), std(점수)))

Accuracy: 0.928 (0.023)


In [18]:
from sklearn.manifold import LocallyLinearEmbedding
LocallyLinearEmbedding(n_components=5).fit_transform(설명변수)

array([[ 0.02760783,  0.00433234, -0.02095877,  0.00680272,  0.04844755],
       [ 0.01966448, -0.00270673,  0.01955879,  0.03550865, -0.02465609],
       [-0.0046547 ,  0.02217439, -0.01581134, -0.01763469,  0.01790108],
       ...,
       [-0.03972864,  0.02724226, -0.00618751, -0.011095  ,  0.01299083],
       [-0.01025056, -0.00886601, -0.00233497,  0.00562289,  0.01698014],
       [ 0.02387398,  0.00062849,  0.00033194,  0.02135728, -0.01534125]])

In [19]:
과정 = [('lle', LocallyLinearEmbedding(n_components=5)), ('m', LogisticRegression())]
모형 = Pipeline(steps=과정)
점수 = cross_val_score(모형, 설명변수, 반응변수, scoring='accuracy',cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(점수), std(점수)))
## 성능이 약간 떨어짐 

Accuracy: 0.876 (0.030)


#### 추가 정보

In [25]:
설명축소 = Isomap(n_components=5).fit_transform(설명변수)
설명축소 = sm.add_constant(설명축소)
print(sm.Logit(반응변수, 설명축소).fit().summary())

Optimization terminated successfully.
         Current function value: 0.185880
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      994
Method:                           MLE   Df Model:                            5
Date:                Mon, 17 Apr 2023   Pseudo R-squ.:                  0.7318
Time:                        16:44:53   Log-Likelihood:                -185.88
converged:                       True   LL-Null:                       -693.14
Covariance Type:            nonrobust   LLR p-value:                4.322e-217
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9161      0.212      4.314      0.000       0.500       1.332
x1             0.4608      0.